![Image](./snapshot_strategy.png)

1차에서 수행한 100개의 질문을 제외하고 다시 랜덤하게 100개 추출


In [1]:
file_pre = '3rd'

In [2]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
# p = p+r'\config'
sys.path.insert(1, p)
sys.path.insert(1, pp)

In [3]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se
import re
import datetime

In [4]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [5]:
q_sql_2223 = """select a.id 
                , a.creationdate
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-12-01' 
              and a.posttypeid = '1'
              and a.tags like '%<python>%' 
              and not exists(select 1 
                                    from tt_posts_difficulty_annotated x 
                                where a.id = x.id)
              and not exists(select 1 
                                    from tt_posts_difficulty_done xx 
                                where a.id = xx.id)
"""

In [6]:
# 2324의 경우 public_for_2324 스키마 tt_posts_difficulty_annotated테이블을 보고 제외
q_sql_2324 = """select a.id 
                , a.creationdate
                , a.title
                , b.body
            from public_for_2324.posts a 
               , public_for_2324.postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2024-12-03' 
              and a.posttypeid = '1'
              and a.tags like '%<python>%' 
              and not exists(select 1 
                                    from public_for_2324.tt_posts_difficulty_annotated x 
                                where a.id = x.id)
              and not exists(select 1 
                                    from tt_posts_difficulty_done xx 
                                where a.id = xx.id)
"""

In [7]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_2223)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [8]:
q_output_for_2223 = pd.DataFrame(rows, columns = [
  'id'
  ,'creationdate'
  ,'title'
  , 'body'
])


In [9]:
q_output_for_2223['22_yn'] = np.where(pd.to_datetime(q_output_for_2223['creationdate']).dt.date < datetime.date(2022, 12, 1), 1, 0)

q_output_for_2223_22 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 1, :]
q_output_for_2223_23 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 0, :]
# first_ann_q_id = np.random.choice(list(q_output['id']), size=30, replace=False)

In [10]:
qid_for_2223_22 = np.random.choice(list(q_output_for_2223_22['id']), size=54, replace=False)
qid_for_2223_23 = np.random.choice(list(q_output_for_2223_23['id']), size=54, replace=False)

In [11]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_2324)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [12]:
q_output_for_2324 = pd.DataFrame(rows, columns = [
  'id'
  ,'creationdate'
  ,'title'
  , 'body'
])


In [13]:
q_output_for_2324['22_yn'] = np.where(pd.to_datetime(q_output_for_2324['creationdate']).dt.date < datetime.date(2022, 12, 1), 1, 0)
q_output_for_2324['23_yn'] = np.where((pd.to_datetime(q_output_for_2324['creationdate']).dt.date < datetime.date(2023, 12, 2)) & (q_output_for_2324['22_yn'] ==0), 1, 0)
q_output_for_2324['24_yn'] = np.where(pd.to_datetime(q_output_for_2324['creationdate']).dt.date >= datetime.date(2023, 12, 2), 1, 0)

print(q_output_for_2324.loc[q_output_for_2324['22_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['22_yn'] ==1, 'creationdate'].max())

print(q_output_for_2324.loc[q_output_for_2324['23_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['23_yn'] ==1, 'creationdate'].max())

print(q_output_for_2324.loc[q_output_for_2324['24_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['24_yn'] ==1, 'creationdate'].max())

# q_output_for_2223_22 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 1, :]
# q_output_for_2223_23 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 0, :]
# first_ann_q_id = np.random.choice(list(q_output['id']), size=30, replace=False)

2021-11-30 00:07:54.443000
2022-11-30 23:58:57.893000
2022-12-01 00:01:48.020000
2023-12-01 23:31:23.847000
2023-12-02 00:14:18.247000
2024-12-02 23:51:31.780000


In [14]:
q_output_for_2324_22 = q_output_for_2324.loc[q_output_for_2324['22_yn'] == 1, :]
q_output_for_2324_23 = q_output_for_2324.loc[q_output_for_2324['23_yn'] == 1, :]
q_output_for_2324_24 = q_output_for_2324.loc[q_output_for_2324['24_yn'] == 1, :]

In [15]:
print(q_output_for_2324_22['creationdate'].min())
print(q_output_for_2324_22['creationdate'].max())

print(q_output_for_2324_23['creationdate'].min())
print(q_output_for_2324_23['creationdate'].max())

print(q_output_for_2324_24['creationdate'].min())
print(q_output_for_2324_24['creationdate'].max())

2021-11-30 00:07:54.443000
2022-11-30 23:58:57.893000
2022-12-01 00:01:48.020000
2023-12-01 23:31:23.847000
2023-12-02 00:14:18.247000
2024-12-02 23:51:31.780000


In [16]:
qid_for_2324_22 = np.random.choice(list(q_output_for_2324_22['id']), size=36, replace=False)
qid_for_2324_23 = np.random.choice(list(q_output_for_2324_23['id']), size=36, replace=False)
qid_for_2324_24 = np.random.choice(list(q_output_for_2324_24['id']), size=36, replace=False)

In [17]:
def chg_tag(code):
    st_pattern = r'<pre(?: class="[^"]*")?><code>'
    st_dst = "```python\n"
    code = re.sub(st_pattern, st_dst, code, count=0, flags=0)
    
    end_dst = "```"
    end_pattern =r'</code></pre>'
    code = re.sub(end_pattern, end_dst, code, count=0, flags=0)
    return code
                                                

In [18]:
def clean_code_df(q_output, q_list):
    q_output_copy = q_output[q_output['id'].isin(q_list)].copy()
    q_output_copy['t_body'] = q_output_copy['body'].apply(chg_tag)
    q_output_copy['clean_body'] = q_output_copy['t_body'].apply(lambda x : htmlp.get_html_cleaned_str(x))
    q_output_copy['clean_body'] = q_output_copy['clean_body'].apply(lambda x:  re.sub(r";(?=\S)", "", x))
    q_output_copy['question'] = """<Title>"""+q_output_copy['title'].map(str)+"""</Title>. <Question>"""+q_output_copy['clean_body'].map(str)+"""</Question> Let's think through the difficulty of question carefully, step by step. """
    return q_output_copy


In [19]:
pp_2324_22 = clean_code_df(q_output_for_2223_22, qid_for_2223_22)
pp_2324_22 = clean_code_df(q_output_for_2223_23, qid_for_2223_23)

pp_2324_22 = clean_code_df(q_output_for_2324_22, qid_for_2324_22)
pp_2324_23 = clean_code_df(q_output_for_2324_23, qid_for_2324_23)
pp_2324_24 = clean_code_df(q_output_for_2324_24, qid_for_2324_24)

In [20]:
snapshop1_sample = pd.concat((pp_2324_22[['id', 'question']], pp_2324_22[['id', 'question']]))
snapshop2_sample = pd.concat((pp_2324_22[['id', 'question']], pp_2324_23[['id', 'question']], pp_2324_24[['id', 'question']]))

In [21]:
snapshop1_sample[['id', 'question']].to_csv(f'./{file_pre}_snapshop1_sample.csv')
snapshop2_sample[['id', 'question']].to_csv(f'./{file_pre}_snapshop2_sample.csv')

## public_for_2324.tt_posts_difficulty_annotated, public.tt_posts_difficulty_annotated 테이블에 인서트 필요

In [23]:
list(snapshop1_sample['id'])

[70306418,
 70325197,
 70448013,
 70556190,
 70811411,
 71102874,
 71261823,
 71389500,
 71594371,
 71674203,
 71710557,
 72118859,
 72422859,
 72735293,
 73030337,
 73708141,
 73964121,
 74316658,
 74553953,
 70217014,
 70266683,
 72329302,
 72705731,
 72773545,
 73281995,
 71143855,
 73784250,
 73672307,
 73764681,
 74280428,
 71797823,
 74324995,
 74514741,
 74483062,
 74507490,
 72842853,
 70306418,
 70325197,
 70448013,
 70556190,
 70811411,
 71102874,
 71261823,
 71389500,
 71594371,
 71674203,
 71710557,
 72118859,
 72422859,
 72735293,
 73030337,
 73708141,
 73964121,
 74316658,
 74553953,
 70217014,
 70266683,
 72329302,
 72705731,
 72773545,
 73281995,
 71143855,
 73784250,
 73672307,
 73764681,
 74280428,
 71797823,
 74324995,
 74514741,
 74483062,
 74507490,
 72842853]